In [1]:
import os

In [2]:
print(os.getcwd())

e:\STUDY\TENSORFLOW\Projects\1_CNN_Project\research


In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'e:\\STUDY\\TENSORFLOW\\Projects\\1_CNN_Project'

In [5]:
# Update the src/cnn_classifier/entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    custom_base_model_path:Path
    training_data:Path
    testing_data:Path
    params_epochs:int
    params_batch_size:int
    params_is_augmentation:bool
    params_image_size:list | tuple
    params_learning_rate:float


In [6]:
# Update the src/cnn_classifier/config/configuration.

from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        # Creating directory
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:

        params=self.params
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        

        training_data=Path(os.path.join(self.config.data_ingestion.unzip_dir, r"data/train") )
        testing_data=Path(os.path.join(self.config.data_ingestion.unzip_dir, r"data/test") )

        create_directories( [Path(training.root_dir)] )

        training_config=TrainingConfig(

            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            custom_base_model_path=Path(prepare_base_model.custom_base_model_path),
            training_data=Path(training_data),
            testing_data=Path(testing_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.INPUT_SHAPE,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config
    

In [8]:
# Update the src/cnn_classifier/components.
import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.keras.utils import image_dataset_from_directory as Images

In [9]:
class Training:

    def __init__(self, config:TrainingConfig):
        self.config=config

    def get_custom_base_model(self):

        self.model=tf.keras.models.load_model( 
                    self.config.custom_base_model_path
                    )
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):

        model.save(path)

    def preprocess_data(self):

        # Load images and split them into batches
        self.images_train = Images(
                            directory=self.config.training_data,
                            labels='inferred',
                            image_size = self.config.params_image_size[:-1],
                            batch_size = self.config.params_batch_size
                            )

        self.images_test = Images(
                            directory=self.config.testing_data,
                            labels='inferred',
                            image_size = self.config.params_image_size[:-1],
                            batch_size = self.config.params_batch_size
                            )
        
    def train(self):

        if self.config.params_is_augmentation:

            data_aug_layers = tf.keras.Sequential(
                [
                    tf.keras.layers.RandomFlip(mode='horizontal_and_vertical' ,
                                               training=True, 
                                               name="Random_flip_layer" ),
                    tf.keras.layers.RandomTranslation(height_factor=(-0.1,.1),
                                                    width_factor=(-0.1,0.1),
                                                    fill_mode="reflect", 
                                                    name= "Random_translation_layers"),
                    tf.keras.layers.RandomRotation(factor=(-0.1,0.1), 
                                                   name="Random_rotation_layer"),
                    tf.keras.layers.RandomZoom(height_factor=(0.1,0.1),
                                               width_factor=(0.1,0.1), 
                                               name="Random_zoom_layer" )
                ],
                name="data_augmentation_layers"
                    )
            
            # Creating new model on top
            inputs=tf.keras.Input(shape=self.config.params_image_size, name="input_layer")

            # Apply random data augmentation
            x = data_aug_layers(inputs )

            Outputs = self.model(x, training=False)

            self.full_model = tf.keras.models.Model(
                                  inputs=inputs,
                                  outputs=Outputs
                                  )

        else :
            self.full_model = self.model

        # Compiling the model
        self.full_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
                                loss="binary_crossentropy",
                                metrics=["accuracy"])
        
        #Fit the model
        self.history_fm = self.full_model.fit(x=self.images_train,
                                              validation_data=self.images_test,
                                              epochs=self.config.params_epochs)


        self.save_model(
                        path=self.config.trained_model_path,
                        model=self.full_model
                       )

  

In [10]:
# Pipeline
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_custom_base_model()
    training.preprocess_data()
    training.train()
    
except Exception as e:
    raise e

[2025-04-25 11:10:25,720 : INFO : common : yaml file : config\config.yaml loaded successfully]
[2025-04-25 11:10:25,720 : INFO : common : yaml file : params.yaml loaded successfully]
[2025-04-25 11:10:25,720 : INFO : common : Created directory at : artifacts]
[2025-04-25 11:10:25,720 : INFO : common : Created directory at : trained_model\training]
Found 369 files belonging to 2 classes.
Found 174 files belonging to 2 classes.
Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 700ms/step - accuracy: 0.6197 - loss: 5.2932 - val_accuracy: 0.8908 - val_loss: 0.7866
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 761ms/step - accuracy: 0.9627 - loss: 0.4146 - val_accuracy: 0.9885 - val_loss: 0.1824
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 23s 956ms/step - accuracy: 1.0000 - loss: 2.5017e-06 - val_accuracy: 0.9770 - val_loss: 0.2110
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 1.0000 - loss: 2.0353e-06 - val_accuracy: 0.9770 - val_loss: 0.2145
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - ac